In [73]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [80]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, classification_report
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')

file = "/Users/mingzhouchen/Desktop/reddit_posts_comments__food.json"
df = pd.read_json(file)

df

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/mingzhouchen/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


id  \
0   1jnoobd   
1   1jngk75   
2   1jns37m   
3   1jnuggm   
4   1jnrrj6   
..      ...   
95  1jo0ms8   
96  1jnqyde   
97  1jnu7f6   
98  1jno97e   
99  1jni0le   

                                                                                                         title  \
0                                                                                     [i ate]Japanese Eel Rice   
1                                                                                        [Homemade] Limoncello   
2                                                                               [Homemade]dutch oven pot roast   
3                                                                              [homemade] Cajun Shrimp Risotto   
4                                                                                  Chicken Shawarma [homemade]   
..                                                                                                         ...   
95                                                                           [I ate] Falafel Kebabs for dinner   
96                                          [homemade] cod poached in coconut milk (two ways) with mango salsa   
97  [homemade] Pollo Roja Taquitos with Black Refried Beans, Guacamole, Spanish Rice, Salsa Roja  and a Salad.   
98                                                                                     [Homemade] Palak Pakodi   
99                        [homemade] taco bell-inspired cheesy gordita crunches and pintos and cheese. 089/365   

                                                                                                                                                                                                                                                                                                                                                                     text  \
0                                                                                                                                                                                                                                                                                                                                                                           
1   From Backyard to Bottle, homemade Limoncello. \n\n1.75ml bottle of everclear, peeled 25 lemons (avoided white piths where possible. Lemon peels tinctured in the everclear for 35 days. Made simple syrup (4 cups water, 2 cups sugar, boiled and let cool) and strained out the peels. I let the infused everclear and simple syrup sit for 7 days before bottling.    
2                                                                                                                                                                                                                                                                                                                                                                           
3                                                                                                                                                                                                                                                                                                                                                                           
4                                                                                                                                                                                                                                                                                                                                                                           
..                                                                                                                                                                                                                                                                           

In [81]:
df = df.astype(str).apply(lambda col: col.str.lower())

df['category'] = df['title'].str.extract(r'\[(.*?)\]')

df['title'] = df['title'].str.replace(r'\[\s*(.*?)\s*\]\s*', '', regex=True)

df.replace(r'^\s*$', "NA", regex=True, inplace = True)

In [82]:
stop_words = set(stopwords.words('english'))

def clean_text(text):
    
    text = re.sub(r'[^a-zA-Z\s]', "", text)
    
    text = ' '.join(word for word in text.split() if word not in stop_words)
    
    return text

df["title"] = df["title"].apply(clean_text)
df["text"] = df["text"].apply(clean_text)
df["comments"] = df["comments"].apply(clean_text)

In [83]:
# 获取停用词
stop_words = set(stopwords.words('english'))

def clean_text(text):
    if not isinstance(text, str):  # 避免处理非字符串数据
        return ""
    
    # 保留重要标点符号，如 ! ? . ，避免影响情感分析
    text = re.sub(r'[^a-zA-Z\s.!?]', '', text)  

    # 移除停用词，但保留否定词 like "not"
    filtered_words = []
    for word in text.split():
        if word.lower() not in stop_words or word.lower() in {"not", "no"}:
            filtered_words.append(word)

    return ' '.join(filtered_words)

# 处理 DataFrame 里的文本
df["title"] = df["title"].apply(clean_text)
df["text"] = df["text"].apply(clean_text)
df["comments"] = df["comments"].apply(clean_text)


In [84]:
df

id  \
0   1jnoobd   
1   1jngk75   
2   1jns37m   
3   1jnuggm   
4   1jnrrj6   
..      ...   
95  1jo0ms8   
96  1jnqyde   
97  1jnu7f6   
98  1jno97e   
99  1jni0le   

                                                                              title  \
0                                                                 japanese eel rice   
1                                                                        limoncello   
2                                                              dutch oven pot roast   
3                                                              cajun shrimp risotto   
4                                                                  chicken shawarma   
..                                                                              ...   
95                                                            falafel kebabs dinner   
96                                    cod poached coconut milk two ways mango salsa   
97  pollo roja taquitos black refried beans guacamole spanish rice salsa roja salad   
98                                                                     palak pakodi   
99                          taco bellinspired cheesy gordita crunches pintos cheese   

                                                                                                                                                                                                                                                                   text  \
0                                                                                                                                                                                                                                                                    NA   
1   backyard bottle homemade limoncello ml bottle everclear peeled lemons avoided white piths possible lemon peels tinctured everclear days made simple syrup cups water cups sugar boiled let cool strained peels let infused everclear simple syrup sit days bottling   
2                                                                                                                                                                                                                                                                    NA   
3                                                                                                                                                                                                                                                                    NA   
4                                                                                                                                                                                                                                                                    NA   
..                                                                                                                                                                                                                                                                  ...   
95                                                                                                                                                                                                                                                                   NA   
96                                                                                                                                                                                                                                                                   NA   
97                                                                                                                                                                                                                painstakingly homemade wasnt following recipe winging   
98                                                                                                               

In [90]:

def get_sentiment_score(text):
    """计算文本的情感得分"""
    blob = TextBlob(text)
    return blob.sentiment.polarity

def classify_sentiment(score):
    """将情感得分转换为类别"""
    return 1 if score > 0 else (-1 if score < 0 else 0)

# 计算 sentiment score
df['comment_sentiment_score'] = df['comments'].apply(get_sentiment_score)
df['text_sentiment_score'] = df['text'].apply(get_sentiment_score)
df['title_sentiment_score'] = df['title'].apply(get_sentiment_score)

# 计算 sentiment category（1: 正面, 0: 中性, -1: 负面）
df['comment_sentiment'] = df['comment_sentiment_score'].apply(classify_sentiment)
df['text_sentiment'] = df['text_sentiment_score'].apply(classify_sentiment)
df['title_sentiment'] = df['title_sentiment_score'].apply(classify_sentiment)

df

id  \
0   1jnoobd   
1   1jngk75   
2   1jns37m   
3   1jnuggm   
4   1jnrrj6   
..      ...   
95  1jo0ms8   
96  1jnqyde   
97  1jnu7f6   
98  1jno97e   
99  1jni0le   

                                                                              title  \
0                                                                 japanese eel rice   
1                                                                        limoncello   
2                                                              dutch oven pot roast   
3                                                              cajun shrimp risotto   
4                                                                  chicken shawarma   
..                                                                              ...   
95                                                            falafel kebabs dinner   
96                                    cod poached coconut milk two ways mango salsa   
97  pollo roja taquitos black refried beans guacamole spanish rice salsa roja salad   
98                                                                     palak pakodi   
99                          taco bellinspired cheesy gordita crunches pintos cheese   

                                                                                                                                                                                                                                                                   text  \
0                                                                                                                                                                                                                                                                    NA   
1   backyard bottle homemade limoncello ml bottle everclear peeled lemons avoided white piths possible lemon peels tinctured everclear days made simple syrup cups water cups sugar boiled let cool strained peels let infused everclear simple syrup sit days bottling   
2                                                                                                                                                                                                                                                                    NA   
3                                                                                                                                                                                                                                                                    NA   
4                                                                                                                                                                                                                                                                    NA   
..                                                                                                                                                                                                                                                                  ...   
95                                                                                                                                                                                                                                                                   NA   
96                                                                                                                                                                                                                                                                   NA   
97                                                                                                                                                                                                                painstakingly homemade wasnt following recipe winging   
98                                                                                                               

In [91]:
unique_categories = df['category'].unique()
print(unique_categories)

category_mapping = {
    'i ate': "Personal Food Experience",
    'homemade': "Homemade Food",
    'pro/chef': "Professional Chef food/Cooking Techniques"
}

df['category_interpretation'] = df['category'].map(category_mapping)

pd.set_option('display.max_colwidth', None)

df.head()

['i ate' 'homemade' 'pro/chef']


id                 title  \
0  1jnoobd     japanese eel rice   
1  1jngk75            limoncello   
2  1jns37m  dutch oven pot roast   
3  1jnuggm  cajun shrimp risotto   
4  1jnrrj6      chicken shawarma   

                                                                                                                                                                                                                                                                  text  \
0                                                                                                                                                                                                                                                                   NA   
1  backyard bottle homemade limoncello ml bottle everclear peeled lemons avoided white piths possible lemon peels tinctured everclear days made simple syrup cups water cups sugar boiled let cool strained peels let infused everclear simple syrup sit days bottling   
2                                                                                                                                                                                                                                                                   NA   
3                                                                                                                                                                                                                                                                   NA   
4                                                                                                                                                                                                                                                                   NA   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [92]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [93]:
# 指定保存路径
output_file = "/Users/mingzhouchen/Desktop/reddit_data.xlsx"

# 将 DataFrame 保存为 Excel 文件
df.to_excel(output_file, index=False, engine="openpyxl")